In [68]:
import pandas as pd
from sodapy import Socrata
import configparser
import json
import os

In [27]:
config = configparser.ConfigParser()
config.read("capstone.cfg")

['capstone.cfg']

In [28]:
app_token = config["CDC"]["APPTOKEN"]

In [29]:
client = Socrata("data.cdc.gov",
                 app_token)


In [30]:

results = client.get("vbim-akqf",
                    limit = 10000)


In [31]:
case_surveillance = pd.DataFrame.from_records(results)

In [32]:
case_surveillance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   cdc_case_earliest_dt     10000 non-null  object
 1   cdc_report_dt            10000 non-null  object
 2   pos_spec_dt              10000 non-null  object
 3   current_status           10000 non-null  object
 4   sex                      10000 non-null  object
 5   age_group                10000 non-null  object
 6   race_ethnicity_combined  10000 non-null  object
 7   hosp_yn                  10000 non-null  object
 8   icu_yn                   10000 non-null  object
 9   death_yn                 10000 non-null  object
 10  medcond_yn               10000 non-null  object
 11  onset_dt                 3720 non-null   object
dtypes: object(12)
memory usage: 937.6+ KB


In [34]:
metadata = client.get_metadata("vbim-akqf")
metadata

{'id': 'vbim-akqf',
 'name': 'COVID-19 Case Surveillance Public Use Data',
 'assetType': 'dataset',
 'attribution': 'CDC Case Surveillance Task Force',
 'attributionLink': 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/index.html',
 'averageRating': 0,
 'category': 'Case Surveillance',
 'createdAt': 1589578317,
 'description': 'Currently, CDC provides the public with COVID-19 case surveillance data in two ways: an 12 data element public use dataset of the line-listed dataset of all COVID-19 cases shared with CDC and a 32 data element <a href="https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Restricted-Access-Detai/mbd7-r32t">restricted access dataset</a> of the line-listed dataset of all COVID-19 cases shared with CDC.\n\nThe following are true for both the public use dataset and the restricted access dataset:\n\n- Data elements can be found on the COVID-19 case report form located at <a href="https://www.cdc.gov/coronavirus/2019-ncov/downloads/pui-form.pdf">

In [37]:
res2 = client.get("vbim-akqf", 
                  group = "sex",
                 select = "sex, count(age_group)")

In [40]:
res2

[{'sex': 'Female', 'count_age_group': '10621441'},
 {'sex': 'Male', 'count_age_group': '9694883'},
 {'sex': 'Missing', 'count_age_group': '50731'},
 {'sex': 'NA', 'count_age_group': '15'},
 {'sex': 'Other', 'count_age_group': '409'},
 {'sex': 'Unknown', 'count_age_group': '197866'}]

In [53]:
res_sex = res2

In [41]:
sum( [ int(row["count_age_group"]) for row in res2])

20565345

In [42]:
!wc ~/Downloads/COVID-19_Case_Surveillance_Public_Use_Data.csv

  20565346  110440471 2416501618 /home/severine/Downloads/COVID-19_Case_Surveillance_Public_Use_Data.csv


In [43]:
res3 = client.get("vbim-akqf", 
            group = "sex, age_group, race_ethnicity_combined, cdc_case_earliest_dt",
            select = "sex, age_group, race_ethnicity_combined, cdc_case_earliest_dt, count(*)"
                 )

In [46]:
res_age = client.get("vbim-akqf", 
                     group = "age_group",
                     select = "age_group, count(*)")

In [50]:
print(len(res_age))
res_age

11


[{'age_group': '0 - 9 Years', 'count': '897435'},
 {'age_group': '10 - 19 Years', 'count': '2172213'},
 {'age_group': '20 - 29 Years', 'count': '3835094'},
 {'age_group': '30 - 39 Years', 'count': '3338670'},
 {'age_group': '40 - 49 Years', 'count': '3051110'},
 {'age_group': '50 - 59 Years', 'count': '2973923'},
 {'age_group': '60 - 69 Years', 'count': '2113504'},
 {'age_group': '70 - 79 Years', 'count': '1172490'},
 {'age_group': '80+ Years', 'count': '848845'},
 {'age_group': 'Missing', 'count': '161984'},
 {'age_group': 'NA', 'count': '77'}]

In [48]:
sum( [ int(row["count"]) for row in res_age])

20565345

In [51]:
res_sex_age = client.get("vbim-akqf", 
                     group = "sex, age_group",
                     select = "sex, age_group, count(*)")

In [54]:
print(len(res_sex_age))
res_sex_age

54


[{'sex': 'Female', 'age_group': '0 - 9 Years', 'count': '429732'},
 {'sex': 'Female', 'age_group': '10 - 19 Years', 'count': '1106527'},
 {'sex': 'Female', 'age_group': '20 - 29 Years', 'count': '2002871'},
 {'sex': 'Female', 'age_group': '30 - 39 Years', 'count': '1709989'},
 {'sex': 'Female', 'age_group': '40 - 49 Years', 'count': '1579131'},
 {'sex': 'Female', 'age_group': '50 - 59 Years', 'count': '1521003'},
 {'sex': 'Female', 'age_group': '60 - 69 Years', 'count': '1067638'},
 {'sex': 'Female', 'age_group': '70 - 79 Years', 'count': '602122'},
 {'sex': 'Female', 'age_group': '80+ Years', 'count': '519717'},
 {'sex': 'Female', 'age_group': 'Missing', 'count': '82711'},
 {'sex': 'Male', 'age_group': '0 - 9 Years', 'count': '452695'},
 {'sex': 'Male', 'age_group': '10 - 19 Years', 'count': '1036909'},
 {'sex': 'Male', 'age_group': '20 - 29 Years', 'count': '1785473'},
 {'sex': 'Male', 'age_group': '30 - 39 Years', 'count': '1588699'},
 {'sex': 'Male', 'age_group': '40 - 49 Years', '

In [55]:
res_sex_age_race = client.get("vbim-akqf", 
                     group = "sex, age_group, race_ethnicity_combined",
                     select = "sex, age_group, race_ethnicity_combined, count(*)")

In [57]:
print(len(res_sex_age_race))
sum( [ int(row["count"]) for row in res_sex_age_race])

378


20565345

In [63]:
res_sex_age_race_date = client.get("vbim-akqf", 
                     group = "cdc_case_earliest_dt, sex, age_group, race_ethnicity_combined",
                     select = "cdc_case_earliest_dt, sex, age_group, race_ethnicity_combined, count(*)",
                                  limit = 200000)

In [65]:
print(len(res_sex_age_race_date))
sum( [ int(row["count"]) for row in res_sex_age_race_date])

88076


20565345

In [60]:
res_date = client.get("vbim-akqf", 
                     group = "cdc_case_earliest_dt",
                      select = "cdc_case_earliest_dt, count(*)")

In [64]:
print(len(res_date))
sum( [ int(row["count"]) for row in res_date])

410


20565345

In [62]:
410 * 378

154980

In [69]:
with open(os.path.join("OUT_DATA", "covid_by_pop_group.json"), "w") as fs :
    json.dump(res_sex_age_race_date, fs)